In [ ]:
%load_ext autoreload
%autoreload 2
import sys, os, time, json, re
import numpy as np
import pandas as pd
from datetime import datetime, timedelta

import data_preprocessing as dp
import backtrader as bt
import matplotlib.pyplot as plt

## Get data

In [ ]:
frequency = timedelta(seconds=60)
pair = 'USDT_BTC'
date_start = '2020-11-11'
date_end = '2021-03-31'
lob_depth = 100
norm_type = 'dyn_z_score'
roll = 1440*10 # 10 days

In [ ]:
df_data, df_data_stdz = dp.import_data(
    pair, 
    date_start, 
    date_end, 
    frequency=frequency, 
    depth=lob_depth, 
    norm_type=norm_type, 
    roll=roll, 
    stdz_depth=1
)

In [ ]:
df_data['Mid_Price'].plot()

## Resample

In [ ]:
# resample data to a less granular frequency
df_data = df_data.asfreq('1min')
df_data['volume'] = df_data['amount_buy'] + df_data['amount_sell']

data_resampled = df_data.resample('30min').agg(
    {
    'Mid_Price': ['last', np.max, np.min], 
    'volume': np.sum
    }
)

data_resampled.columns = data_resampled.columns.get_level_values(1)

data_resampled['close'] = data_resampled['last']
data_resampled['open'] = data_resampled['last'].shift()
data_resampled['high'] = data_resampled['amax']
data_resampled['low'] = data_resampled['amin']
data_resampled['volume'] = data_resampled['sum']
data_resampled.index.name = 'datetime'

data_resampled
# rename columns

In [ ]:
data_resampled['log_ret'] = (np.log(data_resampled['close']) - np.log(data_resampled['close'].shift(1)))
data_resampled['roll_std'] = data_resampled['log_ret'].rolling(window=336).std() # 336 is the number of 30mins interval in week
data_resampled['roll_std'].plot(figsize=(8,4))

## Backtrader

In [ ]:
from Strategies.GoldenCross import GoldenCross
from Strategies.BuyHold import BuyHold

# Create a cerebro entity
cerebro = bt.Cerebro()

# Add a strategy
cerebro.addstrategy(GoldenCross)

# Create a Data Feed
data = bt.feeds.PandasData(dataname=data_resampled[:2000])

# Add the Data Feed to Cerebro
cerebro.adddata(data)

cerebro.addwriter(bt.WriterFile, out='./Strategies/logging/golden_cross2.csv', csv=True)

# Set our desired cash start
cerebro.broker.setcash(200000.0)
# Add a FixedSize sizer according to the stake
# cerebro.addsizer(bt.sizers.PercentSizer, percents=10)
# cerebro.broker.setcommission(commission=0.0007) 

# Print out the starting conditions
print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

# Run over everything


cerebro.run()

plt.rcParams['figure.figsize']=[22, 16]
cerebro.plot()
# Print out the final result
print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())

# figure out what's wrong with stop losses


In [ ]:
strategy_results = pd.read_csv('./Strategies/logging/golden_cross2.csv', header=1, index_col='Id').dropna(thresh=3)
strategy_results['datetime'] = pd.to_datetime(strategy_results['datetime'])
print(strategy_results.shape)

In [ ]:
strategy_and_indic = pd.merge(data_resampled, strategy_results, left_index=True, right_on='datetime', how='outer')
print(strategy_and_indic.columns)
columns_to_keep = ['datetime', 'open_x', 'close_x', 'high_x', 'low_x', 'cash', 'value', 'buy', 'sell', 'pnlplus', 'pnlminus', 'sma', 'sma.1', 'crossover']
strategy_and_indic[columns_to_keep].to_csv('./Strategies/logging/golden_cross_cl.csv')

In [ ]:
# def saveplots(cerebro, numfigs=1, iplot=True, start=None, end=None,
#              width=16, height=9, dpi=300, tight=True, use=None, file_path = '', **kwargs):

#         from backtrader import plot
#         if cerebro.p.oldsync:
#             plotter = plot.Plot_OldSync(**kwargs)
#         else:
#             plotter = plot.Plot(**kwargs)

#         figs = []
#         for stratlist in cerebro.runstrats:
#             for si, strat in enumerate(stratlist):
#                 rfig = plotter.plot(strat, figid=si * 100,
#                                     numfigs=numfigs, iplot=iplot,
#                                     start=start, end=end, use=use)
#                 figs.append(rfig)

#         for fig in figs:
#             for f in fig:
#                 f.savefig(file_path, bbox_inches='tight')
#         # return figs

# saveplots(cerebro, file_path = 'savefig.png') 

## My trading functions

In [ ]:
import ccxt
import ta
from ta.volatility import BollingerBands, AverageTrueRange
import config
import pandas as pd
import plotly.graph_objects as go

In [ ]:
exchange = ccxt.binance(
    {
        'apiKey': config.BINANCE_API_KEY,
        'secret': config.BINANCE_SECRET_KEY
    }
)

markets = exchange.load_markets()

bars = exchange.fetch_ohlcv('ETH/USDT', limit=20) # most recent candle keeps evolving

In [ ]:
df =data_resampled[['open', 'high', 'low', 'close', 'volume']].copy() # pd.DataFrame(bars, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])

bb_indicator = BollingerBands(df['close'], window=20)

df['upper_band'] = bb_indicator.bollinger_hband()
df['lower_band'] = bb_indicator.bollinger_lband()
df['moving_average'] = bb_indicator.bollinger_mavg()

In [ ]:
atr_indicator = AverageTrueRange(df['high'], df['low'], df['close'])
df['atr'] = atr_indicator.average_true_range()

df


In [ ]:
fig = go.Figure(data=[go.Candlestick(x=df.index,
                open=df['open'],
                high=df['high'],
                low=df['low'],
                close=df['close'],
                name='px')])

fig.update_layout(
    xaxis_rangeslider_visible=False,
    width=1400,
    height=600    
)

fig.add_scatter(x=df.index, y=df['atr'], name='atr')
fig.add_scatter(x=df.index, y=df['moving_average'], name='ma')
fig.add_scatter(x=df.index, y=df['upper_band'], name='bb_upper')
fig.add_scatter(x=df.index, y=df['lower_band'], name='bb_lower')

# fig.update_layout(
#     title='The Great Recession',
#     yaxis_title='AAPL Stock',
#     shapes = [dict(
#         x0='2016-12-09', x1='2016-12-09', y0=0, y1=1, xref='x', yref='paper',
#         line_width=2)],
#     annotations=[dict(
#         x='2016-12-09', y=0.05, xref='x', yref='paper',
#         showarrow=False, xanchor='left', text='Increase Period Begins')]
# )

fig.show()